# Logistic Regression
Let's try to implement a logistic regression fit using neural network approach.

In [1]:
# setup
import numpy as np

In [2]:
# sigmoid
def sigmoid(x):
    return(1 / (1 + np.exp(-x)))

# test
sigmoid(np.array([0,1,2,3]))

array([0.5       , 0.73105858, 0.88079708, 0.95257413])

In [3]:
# let's generate some simulated data

# input
n_trials = 1000
x = np.random.uniform(size=n_trials, low=-5, high = 5).reshape((1,n_trials))

# train output
y = np.random.binomial(1,sigmoid(0.8*x-0.5))

print(np.shape(x))
print(x)
print(sigmoid(0.8*x+0.5))
print(y)

(1, 1000)
[[-3.12643829  2.91336456  0.54591639  1.18059691 -4.16889488  4.54695762
   4.9343231   4.2415954  -0.8008969   3.43195774  1.94213274 -1.286525
   1.05576482 -2.32986994  0.6713469   2.99570752  1.2799046   0.2000388
   3.12692811 -1.09894278  1.96132432  0.7251028   4.49936923 -4.2706899
  -3.96257252 -4.50183579 -3.03615164  0.9684484   4.83849606  2.90223034
  -4.38501728 -3.0476561   2.47487892 -1.3260976   3.45634754  0.35963167
   4.9137889  -0.97121066 -4.97224815  0.1653576   3.89399557 -0.22596281
  -2.88023054  3.86847881 -2.60212606 -4.96221825  0.34633483 -1.71269199
   4.15118684 -1.27509967 -4.26842618 -1.39198263 -0.73886721  1.15443897
  -2.0553679  -4.98349493  4.97530939 -4.10495971  3.87391517  4.90084422
   0.30250501  1.16201057  2.96715704  4.02418212  4.88279328 -4.60891563
  -4.45173145  4.4148061   3.24397135 -3.85265581  2.99347738  4.97403656
  -1.16373908 -3.88161151  1.08374901  1.24020616 -2.30933201  0.699802
  -4.30657324  2.73913327 -0.16863

In [ ]:
# z_i = w*x_i + b
# z = w*x + b
# dz/dw = x
# dz/db = 1
# a = yhat = sigmoid(z)
# da/dz = a*(1-a)

# likelihood - want to maximize that
# l = a^y * (1-a)^(1-y)

# loss function - want to minimize hence "minus"
# -log(l) = -( y*log(a) + (1-y)*log(1-a) )

# cost function
# L = -1/m * sum(log(l))

# gradient descent in the notation of NN back propagation
# dz := dL/dz = dL/da*da/dz = -1/m * sum( (y/a + (y-1)/(1-a))*a*(1-a) ) = -1/m * sum(y-a)
# dw := dL/dw = dL/dz * dz/dw := dz * x
# db := dL/db = dL/dz * dz/db := dz
# step
# w -> w - alpha * x * dz
# b -> b - alpha * dz

In [5]:
# starting point
alpha=1
m = np.shape(x)[1]
w = np.ones((1,1))
#w = 1
b = 2
print(m)
print('learning rate:',alpha)
np.shape(w)

1000
learning rate: 1


(1, 1)

In [6]:
dz=1
i=0
w = np.random.uniform(high=10,low=-10,size=1).reshape((1,1))
b = np.random.uniform(high=10,low=-10,size=1)
print(m,w,b)
while(abs(dz)>0.0000001):
#for i in range(1000):
    i += 1
    
    # forward step
    z = np.dot(w.T,x) + b
    A = sigmoid(z)
    cost = -1./m * (np.dot(y,np.log(A).T) + np.dot((1-y),np.log(1-A).T))
    
    # back propagation
    dz = 1./m * np.sum(A-y)
    dw = 1./m * np.dot(x,(A-y).T) 
    db = dz
    
    w = w - alpha * dw
    b = b - alpha * db

    if i==0:
        print('z:',z)
        print('dz:',dz)
        print('dw:',dw)
    
    if i>100000:
        break
    
print(i, dz)
w,b

1000 [[-6.35794674]] [ 6.69886699]
141 9.61773037647e-08


/home/kukarzev/venv_flask/lib/python3.5/site-packages/ipykernel/__main__.py:13: RuntimeWarning: divide by zero encountered in log


(array([[ 0.73347818]]), array([-0.48646616]))

In [7]:
import pandas as pd
df = pd.DataFrame({'x':x[0], 'y':y[0]})
df.head()

,x,y
0,-3.351105,0
1,0.328754,1
2,3.208982,1
3,-3.327454,1
4,4.608755,1


In [9]:
# compare to sklearn
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression

yy, XX = dmatrices('y ~ x', df, return_type = 'dataframe')

model = LogisticRegression(max_iter=100000,tol=0.0000001)
model.fit(XX,yy)
model.coef_

/home/kukarzev/venv_flask/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[-0.24179933,  0.73189205]])

In [10]:
# check with statsmodels
import statsmodels.discrete.discrete_model as sm
logit = sm.Logit(yy, XX)
logit.fit().params

Optimization terminated successfully.
         Current function value: 0.395426
         Iterations 7


Intercept   -0.486467
x            0.733478
dtype: float64